# Get unfinished/problematic events

This script will get the last few unfinished events (if any), from the cameras. 
Will also get the logs for each.

In [ ]:
import boto3
from boto3.dynamodb.conditions import Key, Attr
from datetime import datetime, timedelta

dynamodb = boto3.resource("dynamodb", region_name="eu-west-1")
table = dynamodb.Table("events")

CAMERAS = [
    "B8A44FE820CC__construction_ptz",
    "B8A44FCDF536__parking_lot",
    "B8A44F976508__panoramic_tree",
    "B8A44FB97C3C__panoramic_garages",
    "B8A44F9C9902__train_rails",
    "B8A44FD014E5__distorted_tree"
]

start_time = (datetime.utcnow() - timedelta(hours=24)).isoformat()
end_time = datetime.utcnow().isoformat()

all_items = []

for camera_id in CAMERAS:
    last_key = None
    while True:
        query_kwargs = {
            "KeyConditionExpression": Key("device_id").eq(camera_id) &
                                     Key("event_timestamp").between(start_time, end_time),
            "FilterExpression": Attr("processing_end_timestamp").not_exists(),
        }

        if last_key:
            query_kwargs["ExclusiveStartKey"] = last_key

        response = table.query(**query_kwargs)
        all_items.extend(response.get("Items", []))
        last_key = response.get("LastEvaluatedKey")
        if not last_key:
            break

    print(f"Found {len(all_items)} events.")

In [ ]:
for item in all_items:
    print(item)

In [ ]:
# for each item append its logs from aws
LOG_GROUP_NAME = "/aws/lambda/event_ai_debug_sagemaker"
logs_client = boto3.client("logs", region_name="eu-west-1")

def get_log_events(log_group, log_stream_name):
    try:
        response = logs_client.get_log_events(
            logGroupName=log_group,
            logStreamName=log_stream_name,
            limit=200,
            startFromHead=True
        )
        messages = [event["message"] for event in response["events"]]
        return "".join(messages)
    except logs_client.exceptions.ResourceNotFoundException:
        return "Log stream not found"
    except Exception as e:
        return f"Error: {e}"

for item in all_items:
    log_stream_name = item.get("lambda_log_stream_name")
    if log_stream_name:
        logs = get_log_events(LOG_GROUP_NAME, log_stream_name)
        item["lambda_logs"] = logs
    else:
        item["lambda_logs"] = "No log stream name"    

In [ ]:
# dump each log into a txt file inside failed_events, each filename should be the device_id  + event_timestamp
for item in all_items:
    device_id = item.get("device_id")
    event_timestamp = item.get("event_timestamp")
    req_id = item.get("lambda_aws_request_id")
    if device_id and event_timestamp:
        filename = f"failed_events/{device_id}_{event_timestamp}.txt"
        with open(filename, "w") as f:
            f.write("Request ID of the failed lambda: " + req_id + "\n\n")
            f.write(item["lambda_logs"])
        print(f"Logs saved to {filename}")
    else:
        print("Missing device_id or event_timestamp for item:", item)

print("All logs saved.")